## **Task 2**

 * I didn't have enough resources to run with GPT 2 embedding size and max sequence length and other parameters. So i reduced it to smaller parameters.
 * Task 2: Learnt about Rotary Positional Embedding(Instead of Positional Embedding) and Group Query Attention built(in place of self attention) corresponding function.


# Rotary Positional Embedding:

Compute the angles:
angles = positions.unsqueeze(−1)/10000^((2⋅(freq//2))/embed_size)
Here, freq is a tensor containing frequencies, usually with values
0,2,4,…,embed_size−2


Compute the sine and cosine components:
embeddings=concatenate(sin(angles),cos(angles))embeddings=concatenate(sin(angles),cos(angles))
The final embeddings tensor will have a shape of (
batch_size,sequence_length,embed_size)(batch_size,sequence_length,embed_size).

In [9]:
import torch
def rotary_embedding(positions, embed_size):
    freq = torch.arange(0, embed_size, 2).float()
    angles = positions.unsqueeze(-1) / torch.pow(10000, (2 * (freq // 2)) / embed_size)
    embeddings = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1)
    return embeddings

# Group Query Attention

In [10]:
import torch
def group_query_attention(x, embed_size, num_groups):
    batch_size, seq_length, _ = x.size()

    # queries
    grouped_queries = x.view(batch_size, seq_length // num_groups, num_groups, embed_size)

    # attention
    attended_groups = []
    for group_idx in range(num_groups):
        queries = torch.nn.Linear(embed_size, embed_size)(grouped_queries[:, :, group_idx, :])
        keys = torch.nn.Linear(embed_size, embed_size)(x)
        values = torch.nn.Linear(embed_size, embed_size)(x)

        attention_scores = torch.einsum('bqe,bte->bqt', queries, keys) / (embed_size ** 0.5)
        attention_weights = F.softmax(attention_scores, dim=-1)

        attended_group = torch.einsum('bqt,bte->bqe', attention_weights, values)
        attended_groups.append(attended_group)

    # Combine group results
    attended_sequence = torch.cat(attended_groups, dim=2)

    return attended_sequence